In [1]:
% matplotlib inline

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from sklearn import tree
from sklearn import ensemble
from sklearn import datasets, linear_model
from sklearn import cross_validation, svm
from sklearn.metrics import confusion_matrix
bank = pd.read_csv("bank.csv")

for column in bank.columns:
    #print type(bank[column][0]) is str
    if( type(bank[column][0]) is str ):
        bank[column] = pd.Categorical.from_array(bank[column]).codes
print bank.shape
bank = bank.dropna()
print bank.shape
bank.head()

(45211, 17)
(45211, 17)


,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,Term deposit?
0,58,4,1,2,0,2143,1,0,2,5,8,261,1,-1,0,3,0
1,44,9,2,1,0,29,1,0,2,5,8,151,1,-1,0,3,0
2,33,2,1,1,0,2,1,1,2,5,8,76,1,-1,0,3,0
3,47,1,1,3,0,1506,1,0,2,5,8,92,1,-1,0,3,0
4,33,11,2,3,0,1,0,0,2,5,8,198,1,-1,0,3,0


In [27]:
y = bank['Term deposit?']
X = bank.drop('Term deposit?',axis = 1)
t = tree.DecisionTreeClassifier(max_leaf_nodes=15)
t = t.fit(X, y)
print t.score(X,y)
print t.classes_
print t.feature_importances_
with open("bank1.dot", "w") as f:
    tree.export_graphviz(t, out_file=f, feature_names=X.columns)
print calculate_cnf(X,'Term deposit?',t,y)
#chyba to najlepsze(albo nie patrz dol) -> najbardziej interesuje nas ci ktorzy wzieli pozyczki -> tu jest sa ze 2 duze grupy gdzie ponad 50% wzielo pozyczke

0.898896286302
[0 1]
[ 0.03734651  0.          0.          0.          0.          0.
  0.02085809  0.          0.03326847  0.          0.10476479  0.57542331
  0.          0.06377384  0.          0.16456499  0.        ]
(array([[13485,   525],
       [ 1095,   790]]), 0.89808115759672857)


In [87]:
y = bank['Term deposit?']
X = bank.drop('Term deposit?',axis = 1)
t = tree.DecisionTreeClassifier(max_leaf_nodes=30)
t = t.fit(X, y)
print t.score(X,y)
with open("bank2.dot", "w") as f:
    tree.export_graphviz(t, out_file=f, feature_names=X.columns)
print calculate_cnf(X,'Term deposit?',t,y)
#ponad polowa jupiiiii!!

0.901838048263
(array([[13116,   687],
       [  856,   959]]), 0.90120373927519526)


In [73]:
y = bank['Term deposit?']
X = bank.drop('Term deposit?',axis = 1)
t = tree.DecisionTreeClassifier(max_depth=3, random_state=0)
t = t.fit(X, y)
print t.score(X,y)
with open("bank5.dot", "w") as f:
    tree.export_graphviz(t, out_file=f, feature_names=X.columns)
#to fajniejsze ma duzo lepszego crossa i duzo gup gdzie spory ulamek toci co wzieli
print calculate_cnf(X,'Term deposit?',t,y)

0.891442348101
(array([[13770,   371],
       [ 1372,   494]]), 0.89111013931405014)


In [5]:
#przed usunieciem cech
def calculate_cnf(data,feature,cls,y = []):
    data['train'] = np.random.uniform(0, 1, len(data))

    data_train = data[data['train'] <= 0.65]
    data_test = data[data['train'] > 0.65]

    if( len(y) ==  0 ):
        y_train = data_train[feature]
        y_test = data_test[feature]
        X_train = data_train.drop(feature, axis = 1)
        X_test = data_test.drop(feature, axis = 1)
    else:
        y_train = y[data['train'] <= 0.65]
        y_test = y[data['train'] > 0.65]
        X_train = data_train
        X_test = data_test

    y_pred = cls.fit(X_train, y_train).predict(X_test)
    cnf_matrix = confusion_matrix(y_test, y_pred)
    return cnf_matrix, cls.score(X_test,y_test)

In [6]:
t = tree.DecisionTreeClassifier(max_depth=3)
scores = cross_validation.cross_val_score(t, X, y, cv=5)
print scores.mean()
print calculate_cnf(X,'Term deposit?',t,y)

0.814157335742
(array([[13581,   278],
       [ 1477,   395]]), 0.88843684444726967)


In [55]:
t = ensemble.RandomForestClassifier(n_estimators=30)
scores = cross_validation.cross_val_score(t, X, y, cv=5)
print scores.mean()
print calculate_cnf(X,'Term deposit?',t,y)

0.710744944415
(array([[13506,   418],
       [ 1167,   662]]), 0.89938424427093255)


In [23]:
t = ensemble.ExtraTreesClassifier(n_estimators=15)
scores = cross_validation.cross_val_score(t, X, y, cv=5)
print scores.mean()
print calculate_cnf(X,'Term deposit?',t,y)

0.469629505755
(array([[13612,   389],
       [ 1443,   428]]), 0.88457661290322576)


In [9]:
t = tree.DecisionTreeClassifier(min_samples_split=22000)
scores = cross_validation.cross_val_score(t, X, y, cv=5)
print scores.mean()
print calculate_cnf(X,'Term deposit?',t,y)

0.822783479189
(array([[13755,     0],
       [ 1856,     0]]), 0.8811094740887836)


In [10]:
y = bank['Term deposit?']
X = bank.drop('Term deposit?',axis = 1)
t = tree.DecisionTreeClassifier(max_leaf_nodes=15)
t = t.fit(X, y)
print t.score(X,y)
print t.classes_
print t.feature_importances_
print calculate_cnf(X,'Term deposit?',t,y)

0.898896286302
[0 1]
[ 0.03734651  0.          0.          0.          0.          0.
  0.02085809  0.          0.03326847  0.          0.10476479  0.57542331
  0.          0.06377384  0.          0.16456499]
(array([[13471,   532],
       [ 1050,   781]]), 0.90008841732979661)


In [11]:
col = X.columns
X = X.drop(col[1],axis = 1)
X = X.drop(col[2],axis = 1)
X = X.drop(col[3],axis = 1)
X = X.drop(col[4],axis = 1)
X = X.drop(col[7],axis = 1)
X = X.drop(col[9],axis = 1)
X = X.drop(col[12],axis = 1)
X = X.drop(col[14],axis = 1)
t = tree.DecisionTreeClassifier(max_leaf_nodes=15)
#print column,t.score(X,y)
scores = cross_validation.cross_val_score(t, X, y, cv=5)
print scores.mean()
t = t.fit(X, y)
print t.feature_importances_
print X.columns
print calculate_cnf(X,'Term deposit?',t,y)

0.749524053531
[ 0.03734651  0.          0.02085809  0.03326847  0.10476479  0.57542331
  0.06377384  0.16456499  0.        ]
Index([u'age', u'balance', u'housing', u'contact', u'month', u'duration',
       u'pdays', u'poutcome', u'train'],
      dtype='object')
(array([[13574,   356],
       [ 1253,   613]]), 0.89813876930868575)


In [12]:
y = bank['Term deposit?']
X = bank.drop('Term deposit?',axis = 1)
X = X.drop('duration',axis=1)
t = tree.DecisionTreeClassifier(max_leaf_nodes=15, max_depth=3 )
t = t.fit(X, y)
print t.score(X,y)
print t.feature_importances_
with open("bank3.dot", "w") as f:
    tree.export_graphviz(t, out_file=f, feature_names=X.columns)
print calculate_cnf(X,'Term deposit?',t,y)

0.892791577271
[ 0.15017816  0.          0.00316275  0.          0.          0.          0.
  0.          0.06788533  0.01268403  0.23375976  0.          0.12990689
  0.00543064  0.39699244]
(array([[13929,   133],
       [ 1584,   306]]), 0.89236459378134403)


In [13]:
y = bank['Term deposit?']
X = bank.drop('Term deposit?',axis = 1)
X = X.drop('duration',axis=1)
t = tree.DecisionTreeClassifier(max_depth=3)
t = t.fit(X, y)
print t.score(X,y)
print t.feature_importances_
with open("bank4.dot", "w") as f:
    tree.export_graphviz(t, out_file=f, feature_names=X.columns)
#bez duration duzo gorzej
print calculate_cnf(X,'Term deposit?',t,y)

0.88624449802
[ 0.26637549  0.          0.          0.          0.          0.          0.
  0.          0.          0.01546997  0.41462667  0.          0.23041973
  0.0096325   0.06347564]
(array([[13956,    77],
       [ 1739,   107]]), 0.88563511556143337)


In [14]:
t = tree.DecisionTreeClassifier(max_depth=3)
scores = cross_validation.cross_val_score(t, X, y, cv=5)
print scores.mean()
print calculate_cnf(X,'Term deposit?',t,y)

0.81866930111
(array([[13930,   199],
       [ 1724,   189]]), 0.88012716618875453)


In [15]:
t = tree.DecisionTreeClassifier(max_depth=3,random_state=0)
print calculate_cnf(bank,'Term deposit?',t)

(array([[13697,   417],
       [ 1328,   535]]), 0.89078049696438633)


In [16]:
t = tree.DecisionTreeClassifier(max_depth=2,random_state=0)
print calculate_cnf(bank,'Term deposit?',t)
# to w sumie tez jest fajne bi w stosunkowo niewielkiej czesci(1/6 danych) znajduje sie ponad polowa tych co brali kredyty

(array([[13664,   295],
       [ 1449,   389]]), 0.8895992910046211)
